In [43]:
import pandas as pd
import os
from graph_tool import Graph, stats, spectral
from netpixi.data.gt import *
import netpixi
import cpnet
import seaborn as sns
from math import sqrt
from matplotlib import rcParams
import statsmodels.formula.api as smf
import numpy as np
from scipy.stats import norm

In [44]:
DATA_DIR = './data/transfers_master/data'
os.listdir(DATA_DIR + '/' + '2014')

['spanish_primera_division.csv',
 'dutch_eredivisie.csv',
 'portugese_liga_nos.csv',
 'french_ligue_1.csv',
 'italian_serie_a.csv',
 'russian_premier_liga.csv',
 'english_premier_league.csv',
 'english_championship.csv',
 'german_bundesliga_1.csv']

In [45]:
def loading_data(DATA_DIR, year):
    
    # Leitura do primeiro dataset para carregar as columns
    dir = "./data/transfers_master/data/"+ year + "/dutch_eredivisie.csv"
    data = pd.read_csv(dir)

    # Loop desconsidera o primeiro elemento (carregado no código acima)
    for league in os.listdir(DATA_DIR + '/' + year)[1:]:
            string = (DATA_DIR + '/' + year + '/' + league)
            data_temp = pd.read_csv(string)
            
            # Concatenação dos dados
            data = pd.concat([data, data_temp])
     
    return(data)

def concat_year(start, end, DATA_DIR, data):
    
    for year in range(start, end+1):
        data = pd.concat([data, loading_data(DATA_DIR, str(year))])
        
    return data

In [46]:
DATA_DIR = './data/transfers_master/data'
YEAR = '1992'

data = loading_data(DATA_DIR, YEAR)

In [47]:
data = concat_year(1993, 2015, DATA_DIR, data)

In [48]:
data_in = data.loc[data["transfer_movement"] == "in"]
data_out = data.loc[data["transfer_movement"] == "out"]
data_in = data_in.reset_index()
data_out = data_out.reset_index()

In [49]:
in_names = list(data_in["club_name"].value_counts().index)
out_names = list(data_in["club_involved_name"].value_counts().index)
name_dic = {}
problems = ["Olympique Marseille", "Olympique Lyon", "Eintracht Braunschweig", "Eintracht Frankfurt"]
for in_str in in_names:
    for out_str in out_names:

        if out_str[:10] in in_str and out_str != in_str:
            if (out_str not in problems) or (in_str not in problems):
#                 print(in_str ,", ", out_str)
                name_dic[out_str] = in_str

In [50]:
for k,v in name_dic.items():
    data_in["club_involved_name"].replace({k: v}, inplace=True)
    
# data_in

In [51]:
# data_pre_gal = data_in[data_in["year"] <= 2003]
# data_pos_gal =  data_in[data_in["year"] > 2003]

In [52]:
times1 = list(data_in["club_name"].value_counts().index)
times2 = list(data_in["club_involved_name"].value_counts().index)
all_teams = times1 + list(set(times2) - set(times1))
len(all_teams)
matriz_dinamica = np.zeros((len(all_teams), len(all_teams)))
all_teams.index("Feyenoord Rotterdam")

0

In [53]:
g2 = Graph(directed=True) # dirigido
g2.vp['name'] = g2.new_vp('object')
g2.vp['league'] = g2.new_vp('object')
g2.ep["transfer"] = g2.new_ep('object')

In [54]:
times = []
for index, rows in data_in.iterrows():
    time1 = rows[1]
    time2 = rows[5]
    matriz_dinamica[all_teams.index(time1)][all_teams.index(time2)] += 1
    
    if time1 not in times:
        times.append(time1)
        g2.add_vertex()
        v_time = g2.vertex(times.index(time1))
        g2.vp['name'][v_time] = time1
        g2.vp['league'][v_time] = rows[10]
        
    if time2 not in times:
        times.append(time2)
        g2.add_vertex()
        v_time = g2.vertex(times.index(time2))
        g2.vp['name'][v_time] = time2
        g2.vp['league'][v_time] = rows[10]
    
    index1 = times.index(time1)
    index2 = times.index(time2)
    
    for i in range(len(all_teams)):
        for j in range(len(all_teams)):
            if matriz_dinamica[i][j]  >= 1:
                if g2.edge(index1, index2) == None:
                    g2.add_edge(index1, index2)
                    e = g2.edge(index1, index2)
                    g2.ep["transfer"][e] = matriz_dinamica[i][j]
                
#     if g2.edge(index1, index2) == None:
#         g2.add_edge(index1, index2)
#         e = g2.edge(index1, index2)
#         g2.ep["transfer"][e] = 1
#     else:
#         e = g2.edge(index1, index2)
#         g2.ep["transfer"][e] = g2.ep["transfer"][e] + 1

# for i in range(len(all_teams)):
#     for j in range(len(all_teams)):
#         if matriz_dinamica[i][j]  >= 1:
#             g2.add_edge(index1, index2)
#             g2.ep["transfer"][e] = 1
            

matriz_dinamica

KeyboardInterrupt: 

In [ ]:
# times = []
# for index, rows in data_in.iterrows():
#     time1 = rows[1]
#     time2 = rows[5]
#     if time1 not in times:
#         times.append(time1)
#         g2.add_vertex()
#         v_time = g2.vertex(times.index(time1))
#         g2.vp['name'][v_time] = time1
#         g2.vp['league'][v_time] = rows[10]
        
#     if time2 not in times:
#         times.append(time2)
#         g2.add_vertex()
#         v_time = g2.vertex(times.index(time2))
#         g2.vp['name'][v_time] = time2
#         g2.vp['league'][v_time] = rows[10]
    
#     index1 = times.index(time1)
#     index2 = times.index(time2)
    
#     if g2.edge(index1, index2) == None:
#         g2.add_edge(index1, index2)
#         e = g2.edge(index1, index2)
#         g2.ep["transfer"][e] = 1
#     else:
#         e = g2.edge(index1, index2)
#         g2.ep["transfer"][e] = g2.ep["transfer"][e] + 1

In [ ]:
stats.remove_self_loops(g2)


In [ ]:

g2.vp['color'] = g2.new_vp('object')

for v in g2.vertices():
    if g2.vp['league'][v] == "Premier League":
        g2.vp['color'][v] = 0x0000ff
        
    if g2.vp['league'][v] == "Eredivisie":
        g2.vp['color'][v] = 0x00ff00
        
    if g2.vp['league'][v] == "Championship":
        g2.vp['color'][v] = 0x0000f0
        
    if g2.vp['league'][v] == "Serie A":
        g2.vp['color'][v] = 0xff00ff
        
    if g2.vp['league'][v] == "Liga Nos":
        g2.vp['color'][v] = 0xff0000
        
    if g2.vp['league'][v] == "Ligue 1":
        g2.vp['color'][v] = 0xffff00
        
    if g2.vp['league'][v] == "1 Bundesliga":
        g2.vp['color'][v] = 0x1100ff
        
    if g2.vp['league'][v] == "Premier Liga":
        g2.vp['color'][v] = 0x00ffff

In [ ]:
def centro_periferia(g, t):
    matrix = spectral.adjacency(g)
    if t == "core":
        algorithm = cpnet.LowRankCore()
    elif t == "coreness":
        algorithm = cpnet.MINRES()

    algorithm.detect(matrix)
    c = algorithm.get_coreness()

    g.vp[t] = g.new_vp('object')
    for i, coreness in c.items():
        v = g.vertex(i)
        g.vp[t][v] = float(coreness)

In [ ]:
# centro_periferia(g2, "core")

In [ ]:
df = pd.DataFrame({k: list(v) for k, v in g2.vp.items()})
df

In [ ]:
def get_weight(grafo, propriedade):
    l = []
    for u in grafo.vertices():
        i = 0
        for e in u.out_edges():
            i += grafo.ep[propriedade][e]
        l.append(i)
    return l

In [ ]:
df["transfer"] = get_weight(g2, "transfer")
df_dicotomizado = df[df["transfer"] > df["transfer"].mean()]

In [ ]:
def get_all_edges(grafo, propriedade):
    edges = []
    for u in grafo.vertices():
        for e in u.out_edges():
            transfer = grafo.ep[propriedade][e]
            edges.append(transfer)
    return edges

In [ ]:
# all_edges = get_all_edges(g2, "transfer")

In [ ]:
stats.remove_parallel_edges(g2)
m = draw_gt.sfdp_layout(g2)
move_gt(g2, m)
save_gt(g2, 'rede_normal.net.gz')

In [ ]:
# def removing_edges(grafo, propriedade, remove_value):
#     for u in grafo.vertices():
#         for e in u.out_edges():
#             transfer = grafo.ep[propriedade][e]
#             if transfer <= remove_value:
#                 grafo.remove_edge(e)
#     return grafo
# rede_dicotomizada = removing_edges(g2, "transfer", 1)

In [ ]:
# stats.remove_parallel_edges(rede_dicotomizada)
# m = draw_gt.sfdp_layout(rede_dicotomizada)
# move_gt(rede_dicotomizada, m)
# save_gt(rede_dicotomizada, 'rede_dicotomizada.net.gz')

In [ ]:
render_dicotomizada = netpixi.render('rede_normal.net.gz')

In [ ]:
render_dicotomizada.graph(color=0x000000)
render_dicotomizada.vertex_default(size=10)
render_dicotomizada.edge_default(color=0xFFFFFF, width=1, curve1=0, curve2=0)